In [1]:
import rasterio
import numpy as np
import laspy
import laszip
from rasterio.transform import from_origin

In [2]:
input_file = 'unclassified_points_with_one_return.las'

In [3]:
with laspy.open(input_file, mode='r') as fh:
    # print amount of points
    las = fh.read()
    print('Points from data:', len(las.points))
    min_boundaing_box_x = min(las['x'])
    min_boundaing_box_y = min(las['y'])
    max_boundaing_box_x = max(las['x'])
    max_boundaing_box_y = max(las['y'])

Points from data: 585287


In [4]:
(min_boundaing_box_x, min_boundaing_box_y), (max_boundaing_box_x, max_boundaing_box_y)

((187465.5, 315228.5), (187965.5, 315728.5))

In [5]:
print(list(las.point_format.standard_dimension_names))

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'gps_time', 'red', 'green', 'blue']


In [6]:
raster_vegetation = np.empty((int((max_boundaing_box_x-min_boundaing_box_x)/0.5), 
                      int((max_boundaing_box_y-min_boundaing_box_y)/0.5)))
raster_vegetation[:] = 0
for n, point in enumerate(las.points):
    x = point.array['X']/ 100
    y = point.array['Y']/ 100
    z = point.array['Z']/ 100
    diff_x = (x - min_boundaing_box_x) * 0.9999999
    diff_y = (y - min_boundaing_box_y) * 0.9999999
    x_index = int(diff_x/0.5)
    y_index = 999 - int(diff_y/0.5)
    if raster_vegetation[y_index, x_index] < z:
        raster_vegetation[y_index, x_index] = z

In [7]:
# Specify the top-left coordinates in the CRS (not latitude and longitude)
top_left_x = min_boundaing_box_x  # Replace with your desired X-coordinate
top_left_y = max_boundaing_box_y  # Replace with your desired Y-coordinate

# Define the pixel size (resolution)
pixel_size_x = 0.5  # Replace with your desired X resolution
pixel_size_y = 0.5  # Replace with your desired Y resolution (negative for north-up)

# Create a transform from the specified parameters
transform = from_origin(top_left_x, top_left_y, pixel_size_x, pixel_size_y)


In [8]:
output_file = "raster_vegetation.tiff"
with rasterio.open(output_file, 'w',
                    driver='GTiff',
                    height=raster_vegetation.shape[0],
                    width=raster_vegetation.shape[1],
                    count=1,
                    dtype=raster_vegetation.dtype,
                    crs=rasterio.crs.CRS.from_epsg(7415),
                    transform=transform) as dst:
            dst.write(raster_vegetation, 1)